<a href="https://colab.research.google.com/github/Jonny55921/CSCI-167/blob/main/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [4]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  query = np.dot(omega_q, x) + beta_q
  key = np.dot(omega_k, x) + beta_k
  value = np.dot(omega_v, x) + beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [5]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  exp_values = np.exp(items_in)
  # Sum over the values
  denom = np.sum(exp_values)
  # Compute softmax
  items_out = exp_values / denom

  return items_out ;

Now compute the self attention values:

In [6]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # Compute the appropriate dot product
    dot_product = np.dot(all_queries[n].T, key)

    # Store dot product
    all_km_qn.append(dot_product[0,0])  # Extract scalar value

  # Compute dot product
  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  print("Attentions for output ", n)
  print(attention)

  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line
  x_prime = np.zeros_like(all_values[0])  # Initialize x_prime

  for i in range(len(all_values)):
    x_prime += attention[i] * all_values[i]

  all_x_prime.append(x_prime)


# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[1.24326146e-13 9.98281489e-01 1.71851130e-03]
Attentions for output  1
[2.79525306e-12 5.85506360e-03 9.94144936e-01]
Attentions for output  2
[0.00505708 0.00654776 0.98839516]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [12]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
    # Get the max value per column for numerical stability
    max_values = np.max(data_in, axis=0)
    shifted_data = data_in - max_values  # Broadcasting subtraction

    # Exponentiate the shifted data
    exp_values = np.exp(shifted_data)

    # Sum over columns (axis 0)
    denom = np.sum(exp_values, axis=0)

    # Normalize by the sum (broadcasted division)
    softmax = exp_values / denom

    return softmax

In [15]:
 # Now let's compute self attention in matrix form
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # Compute queries, keys, and values
    Queries = np.dot(omega_q, X) + beta_q
    Keys = np.dot(omega_k, X) + beta_k
    Values = np.dot(omega_v, X) + beta_v

    # Compute dot products (unnormalized attentions)
    DotProducts = np.dot(Queries.T, Keys)

    # Scale the dot product (for stability in softmax)
    D_k = Keys.shape[0]  # Dimensionality of keys
    ScaledDotProduct = DotProducts / np.sqrt(D_k)

    # Apply softmax to calculate attentions
    Attentions = softmax_cols(ScaledDotProduct)

    # Weight values by attentions (Corrected)
    X_prime = np.dot(Values, Attentions)

    return X_prime

In [16]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 1.64608331  0.66149223  1.21854105]
 [-0.08376846  0.36015254 -0.17896151]
 [ 4.05660686 -0.05852563  1.02632615]
 [ 2.20233961  0.09885586  0.58832268]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [17]:
# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # 1. Compute queries, keys, and values
    Queries = np.dot(omega_q, X) + beta_q  # Shape: (N, D_q)
    Keys = np.dot(omega_k, X) + beta_k    # Shape: (N, D_k)
    Values = np.dot(omega_v, X) + beta_v  # Shape: (N, D_v)

    # 2. Compute dot products (unnormalized attention scores)
    DotProducts = np.dot(Queries.T, Keys)  # Shape: (D_q, N) . (N, D_k) => (D_q, D_k)

    # 3. Scale the dot products (as in equation 12.9)
    D_k = omega_k.shape[0]  # Dimensionality of the key vectors
    ScaledDotProducts = DotProducts / np.sqrt(D_k)  # Scaling factor

    # 4. Apply softmax to calculate attentions (normalize by columns)
    Attentions = softmax_cols(ScaledDotProducts)

    # 5. Weight values by attentions (final weighted sum)
    X_prime = np.dot(Values, Attentions)  # Shape: (D_v, N) . (N, N) => (D_v, N)

    return X_prime

In [18]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 1.64608331  0.66149223  1.21854105]
 [-0.08376846  0.36015254 -0.17896151]
 [ 4.05660686 -0.05852563  1.02632615]
 [ 2.20233961  0.09885586  0.58832268]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.

In the code below, we reuse our softmax and attention functions as well as a sample array to check for covariance with permutations. As you can see below after permutating the array, covariance changes. The values between the two final arrays are quite different as we can see.  


In [22]:
import numpy as np

# Define a simple permutation matrix for testing (example: swap columns 1 and 3)
P = np.array([[0, 0, 1],
              [1, 0, 0],
              [0, 1, 0]])

# Define a random input matrix X (e.g., 3 examples, 3 features)
X = np.random.randn(3, 3)

# Define omega matrices and beta vectors (e.g., randomly)
omega_v = np.random.randn(3, 3)
omega_q = np.random.randn(3, 3)
omega_k = np.random.randn(3, 3)
beta_v = np.random.randn(3)
beta_q = np.random.randn(3)
beta_k = np.random.randn(3)

# Define the self-attention function (scaled dot product as defined earlier)
def softmax_cols(data_in):
    max_values = np.max(data_in, axis=0)  # Get max for numerical stability
    shifted_data = data_in - max_values  # Shift data
    exp_values = np.exp(shifted_data)  # Exponentiate the shifted data
    denom = np.sum(exp_values, axis=0)  # Sum over columns
    softmax = exp_values / denom  # Normalize by the sum
    return softmax

def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    Queries = np.dot(omega_q, X) + beta_q  # Shape: (N, D_q)
    Keys = np.dot(omega_k, X) + beta_k    # Shape: (N, D_k)
    Values = np.dot(omega_v, X) + beta_v  # Shape: (N, D_v)
    DotProducts = np.dot(Queries.T, Keys)  # Shape: (D_q, N) . (N, D_k) => (D_q, D_k)
    D_k = omega_k.shape[0]  # Dimensionality of the key vectors
    ScaledDotProducts = DotProducts / np.sqrt(D_k)  # Scaling factor
    Attentions = softmax_cols(ScaledDotProducts)
    X_prime = np.dot(Values, Attentions)  # Shape: (D_v, N) . (N, N) => (D_v, N)
    return X_prime

# Apply attention to original X
X_prime = scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Permute the columns of X using matrix P
X_permuted = np.dot(P, X)

# Apply attention to permuted X
X_prime_permuted = scaled_dot_product_self_attention(X_permuted, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

print("Original Input Matrix X:")
print(X)
print("\nPermuted Input Matrix X_P (using permutation matrix P):")
print(X_permuted)

# Print the outputs from self-attention
print("\nOutput of self-attention on original X (X'):")
print(X_prime)

print("\nOutput of self-attention on permuted X (X_P'):")
print(X_prime_permuted)


Original Input Matrix X:
[[-0.17154633  0.77179055  0.82350415]
 [ 2.16323595  1.33652795 -0.36918184]
 [-0.23937918  1.0996596   0.65526373]]

Permuted Input Matrix X_P (using permutation matrix P):
[[-0.23937918  1.0996596   0.65526373]
 [-0.17154633  0.77179055  0.82350415]
 [ 2.16323595  1.33652795 -0.36918184]]

Output of self-attention on original X (X'):
[[-4.09453714 -3.55665784 -2.77267499]
 [ 0.26199143 -0.05176547 -0.41585387]
 [-0.15097688  0.30390862  0.86032896]]

Output of self-attention on permuted X (X_P'):
[[-0.42299352 -0.48805159 -0.71898878]
 [-0.57817855 -0.64222301 -0.44361882]
 [ 0.93680422  1.01310935  1.26948369]]
